<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/ARAFS/prep_data/download_CMEMS_SATdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downlod data from the Copernicus Marine Service (CMS).
- Best to check following URLs to make sure dataset IDs are correct.
  - [OSTIA Sea Surface Temperature (SST).](https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_L4_NRT_008_046/services)
  - [AVISO Sea Surface Height (SSH).](https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_L4_NRT_008_046/services)
  - [CNS Sea Surface Salinity (SSS).](https://data.marine.copernicus.eu/product/MULTIOBS_GLO_PHY_S_SURFACE_MYNRT_015_013/description)

# Inputs

- Start and end dates of `case 1`: Dec, 2022- Jan, 2023.
- CMS `user name` and `password`; create it if needed.
- Names/details of CMEMS datasets may need edits; see below.

In [ ]:
!pip install --upgrade copernicusmarine

In [ ]:
import os

import copernicusmarine

import numpy as np
import pandas as pd
import xarray as xr

from datetime import date, timedelta

In [ ]:
def get_cmems_data(dsetID, vNames, lon_start, lon_end, lat_start, lat_end, time_start, time_end):

  data_request = {"dataset_id" : dsetID,
    "longitude" : [lon_start, lon_end],
    "latitude" : [lat_start, lat_end],
    "time" : [time_start, time_end],
    "variables" : vNames}

  cms_data =copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"])

  #print(cms_data)

  return cms_data

In [ ]:
# Start and end dates of `case 1`

start_date = date(2022, 12, 20)
end_date = date(2023, 1, 20)

lon_min, lon_max = [-179, -90] # deg West
lat_min, lat_max = [10, 75] # deg North

In [ ]:
# Set Copernicus credentials
CMEMS_username, CMEMS_passwd = ["sakella", "HbFPyP9M"]
copernicusmarine.login(username=CMEMS_username, password=CMEMS_passwd)

In [ ]:
drive_path = '/content/drive/MyDrive/UFS-no-RTOFS/AR/work/data/'
os.makedirs(drive_path, exist_ok=True)

In [ ]:
# CMEMS dataset IDs

# AVISO L4 SSH (ADT)/SLA/Geostrophic currents
def aviso_l4_ssh(year):

  if year < 2024:
    dsetID = "cmems_obs-sl_glo_phy-ssh_nrt_allsat-l4-duacs-0.25deg_P1D"
    vNames = ["adt", "sla", "err_sla", "ugos", "vgos"]
  else:
    dsetID = "cmems_obs-sl_glo_phy-ssh_nrt_allsat-l4-duacs-0.125deg_P1D"
    vNames = ["adt", "sla", "err_sla", "ugos", "vgos"]

  return dsetID, vNames

# OSTIA SST
sst={'dsetID':'METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2',
     'vNames':['analysed_sst'],
     'varName': 'SST'}

# CNS SSS NRT
sss={'dsetID':'cmems_obs-mob_glo_phy-sss_nrt_multi_P1D',
     'vNames':['sos', 'dos'],
     'varName': 'SSS'}

# Download data

In [ ]:
# AVISO SSH
# p1: 2022/12/20 (start date)- 2022/12/31
# p2: 2023/01/01- end date
# because of dataset name dependence on `year`, doing in 2 parts, can be done elegantly. LATER!

ssh_data_p1 = get_cmems_data(*aviso_l4_ssh(start_date.year),
                        lon_min, lon_max, lat_min, lat_max,\
                        start_date.strftime('%Y-%m-%d'),\
                        '2022-12-31')

ssh_data_p2 = get_cmems_data(*aviso_l4_ssh(end_date.year),
                        lon_min, lon_max, lat_min, lat_max,\
                        '2023-01-01',\
                        end_date.strftime('%Y-%m-%d'))

ssh_data = xr.concat([ssh_data_p1, ssh_data_p2], dim='time')

# add geostrophic currents to the dataset- eases our life!
ssh_data['surf_curr'] =\
xr.DataArray(np.sqrt(ssh_data.ugos**2 + ssh_data.vgos**2),\
coords=ssh_data.ugos.coords, dims=ssh_data.ugos.dims,\
             name='surf_curr', attrs={'units':'m/s'})

ssh_data.drop_vars(['ugos', 'vgos'])

output_filename = f'aviso_ssh_case1.nc'
output_path = drive_path + output_filename
ssh_data.to_netcdf(output_path)
print(f"Saved AVISO SSH data to {output_path}\n")

In [ ]:
# download other dataset(s)
for dtype in [sst, sss]:
  sat_data = get_cmems_data(dtype['dsetID'], dtype['vNames'],\
                            lon_min, lon_max, lat_min, lat_max,\
                            start_date.strftime('%Y-%m-%d'),\
                            end_date.strftime('%Y-%m-%d'))

  output_filename = f'{dtype["varName"]}_case1.nc'
  output_path = drive_path + output_filename
  sat_data.to_netcdf(output_path)
  print(f"Saved {dtype} data to {output_path}\n")

## Make a few movies!

In [ ]:
!pip install xmovie

In [ ]:
from shapely.errors import ShapelyDeprecationWarning
from xmovie import Movie

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [ ]:
ssh_data = xr.open_dataset(drive_path + 'aviso_ssh_case1.nc')
print(f"Read SSH data\n")

In [ ]:
subset_ssh = ssh_data.sel(longitude=slice(-140, -100), latitude=slice(20, 50))
ssh_mov = Movie(subset_ssh.sla, vmin=-0.1, vmax=0.4)

ssh_mov.save('AVISO_SLA_case1.gif', remove_movie=False, progress=True, framerate=5,gif_framerate=5)

!rm -f *.mp4

In [ ]:
sst_filename =  drive_path + 'SST_case1.nc'
sst_data = xr.open_dataset(sst_filename)
print(f"Read SST data from {sst_filename}\n")

subset_sat = sst_data.sel(longitude=slice(-140, -110), latitude=slice(15, 45))
mov = Movie((subset_sat['analysed_sst']-273.15), vmin=10., vmax=25, cmap='jet')

mov.save('OSTIA_SST_case1.gif', remove_movie=False, progress=True, framerate=5,gif_framerate=5)

!rm -f *.mp4

In [ ]:
sss_filename =  drive_path + 'SSS_case1.nc'
sss_data = xr.open_dataset(sss_filename)
print(f"Read SSS data from {sss_filename}\n")

subset_sat = sss_data.sel(longitude=slice(-140, -110), latitude=slice(15, 45))

# surface density
#mov = Movie((subset_sat['dos']-1000), vmin=21., vmax=26, cmap='gist_ncar')
#mov.save('CNS_DOS_case1.gif', remove_movie=False, progress=True, framerate=5,gif_framerate=5)

# SSS
mov = Movie((subset_sat['sos']), vmin=31., vmax=35, cmap='gist_ncar')
mov.save('CNS_SSS_case1.gif', remove_movie=False, progress=True, framerate=5,gif_framerate=5)

!rm -f *.mp4

# Slice along `longitude`

In [ ]:
lon_stn = np.arange(-140, -115, 5)

lat_s, lat_e = [10, 30]
print(lon_stn)

# Plot ADT

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)

for ilon, lon_s in enumerate(lon_stn):
  print(ilon, lon_s)
  ssh_data.sel(longitude=lon_s, method='nearest').sel(latitude=slice(lat_s, lat_e)).mean(('latitude')).adt.plot(x='time', ax=ax, label=f'{lon_s}')

ax.legend(loc=1)
ax.set_title(f'Lat ($^\circ$N): {lat_s}- {lat_e}')
ax.set_ylabel('ADT [m]')

# Plot SST

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)

for ilon, lon_s in enumerate(lon_stn):
  #print(ilon, lon_s)
  sst_data.sel(longitude=lon_s, method='nearest').sel(latitude=slice(lat_s, lat_e)).mean(('latitude')).analysed_sst.plot(x='time', ax=ax, label=f'{lon_s}')

ax.legend(loc=1)
ax.set_title(f'Lat ($^\circ$N): {lat_s}- {lat_e}')
ax.set_ylabel('SST [K]')

# Plot SSS and Surface Density

In [ ]:
fig = plt.figure(figsize=(18, 4))

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for ilon, lon_s in enumerate(lon_stn):
  #print(ilon, lon_s)
  sss_data.sel(longitude=lon_s, method='nearest').sel(latitude=slice(lat_s, lat_e)).mean(('latitude')).sos.plot(x='time', ax=ax1, label=f'{lon_s}')

  sss_data.sel(longitude=lon_s, method='nearest').sel(latitude=slice(lat_s, lat_e)).mean(('latitude')).dos.plot(x='time', ax=ax2, label=f'{lon_s}')

ax1.legend(loc=1)
ax1.set_title(f'Lat ($^\circ$N): {lat_s}- {lat_e}')
ax1.set_ylabel('SSS [PSU]')

ax2.legend(loc=2, ncol=2)
ax2.set_title(f'Lat ($^\circ$N): {lat_s}- {lat_e}')
ax2.set_ylabel('SSD [kg/m$^3$]')